In [1]:
import numpy as np
import sys
sys.executable
from os.path import join
from smpl_webuser.serialization import load_model
from fitting.util import write_simple_obj, safe_mkdir
import trimesh
import sklearn
import pyrender
from sklearn.metrics import mean_squared_error
import torch
import glob

#from rlagent import PGAgent
# Load FLAME model (here we load the female model)
# Make sure path is correct
female_path = './models/female_model.pkl'
male_path = './models/male_model.pkl'
generic_path = './models/generic_model.pkl'
model = load_model(generic_path)
print ("loaded model from:", male_path)
variables = [i for i in dir(model) if not callable(i)]
print(variables)
print(type(model.posedirs))

('loaded model from:', './models/male_model.pkl')
['J', 'J_regressor', 'J_transformed', 'T', '__abs__', '__add__', '__array__', '__array_priority__', '__call__', '__class__', '__delattr__', '__dict__', '__div__', '__doc__', '__float__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__int__', '__le__', '__len__', '__lt__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdiv__', '__reduce__', '__reduce_ex__', '__repr__', '__rmul__', '__rpow__', '__rsub__', '__setattr__', '__setitem__', '__setstate__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__truediv__', '__weakref__', '_cache', '_cached_parms', '_call_on_changed', '_compute_dr_wrt_sliced', '_default_kwargs', '_depends_on_deps', '_dirty_vars', '_invalidate_cacheprop_names', '_itr', '_make_dense', '_make_sparse', '_parents', '_reserved_kw', '_setup_terms', '_status', '_superdot', 'a', 'add_dterm', 'b'

In [2]:
print "\nFLAME coefficients:"
print "shape (identity) coefficient shape =", model.betas[0:300].shape # valid shape component range in "betas": 0-299
print "expression coefficient shape       =", model.betas[300:].shape  # valid expression component range in "betas": 300-399
print "pose coefficient shape             =", model.pose.shape

print "\nFLAME model components:"
print "shape (identity) component shape =", model.shapedirs[:,:,0:300].shape
print "expression component shape       =", model.shapedirs[:,:,300:].shape
print "pose corrective blendshape shape =", model.posedirs.shape
print ""


FLAME coefficients:
shape (identity) coefficient shape = (300,)
expression coefficient shape       = (100,)
pose coefficient shape             = (15,)

FLAME model components:
shape (identity) component shape = (5023, 3, 300)
expression component shape       = (5023, 3, 100)
pose corrective blendshape shape = (5023, 3, 36)



In [3]:
model.pose[0:15] = 0
mesh = trimesh.Trimesh(model.r, model.f)
A = np.array(model.shapedirs)
mesh.show(viewer = 'gl')

SceneViewer(width=1299, height=713)

In [4]:
#changing model parameters
#model.pose[0:15] = 1
#mesh = trimesh.Trimesh(model.r, model.f)
#mesh.show(viewer = 'gl')

In [5]:
obj_load = trimesh.load_mesh('/home/michalislazarou/PhD/VOCA/voca/animation_output/00087.obj')
#obj_load.show(viewer = 'gl')
loss = sum(sum((obj_load.vertices - model.r)**2))
print(loss)

71.36866434962708


In [19]:
print(loss1, model.r[0], mesh.vertices[0])
print(mesh.vertices[0] - model.r[0])

(0.0044899399101906295, array([-0.01754271, -0.08577443,  0.10695862]), TrackedArray([ 0.07409152, -0.22313159,  0.07183197]))
[ 0.09163424 -0.13735716 -0.03512665]


In [4]:
obj_load = trimesh.load_mesh('/home/michalislazarou/PhD/VOCA/voca/animation_output/00191.obj')
mesh = pyrender.Mesh.from_trimesh(obj_load)
scene = pyrender.Scene()
#variables = [i for i in dir(scene) if not callable(i)]
#camera = pyrender.IntrinsicsCamera(fx, fy, cx, cy, yfov = 1, aspectRatio=1.0, znear=0.05 )
#r = pyrender.OffscreenRenderer(viewport_width=640, viewport_height=480, point_size=1.0)

#print(variables)
#v = pyrender.Viewer(scene, use_raymond_lighting=True, run_in_thread=True)
#for name in sorted(glob.glob('/home/michalislazarou/PhD/VOCA/voca/animation_output/*.obj')):
 #   obj_load = trimesh.load_mesh(name)
  #  mesh = pyrender.Mesh.from_trimesh(obj_load)
scene.add(mesh)
#scene.add(camera)

pyrender.Viewer(scene, use_raymond_lighting=True)

Viewer(width=640, height=480)

In [ ]:
model.pose[:]  = np.random.randn( model.pose.size ) * 0.05
model.betas[:] = np.random.randn( model.betas.size ) * 1.0
print(model.pose)
print(model.r.shape)
path = '/home/michalislazarou/PhD/VOCA/voca/animation_output'
#mesh = trimesh.load_mesh('/home/michalislazarou/PhD/VOCA/voca/animation_output/00191.obj')
for name in sorted(glob.glob('/home/michalislazarou/PhD/VOCA/voca/animation_output/*.obj')):
    mesh = trimesh.load_mesh(name)
    #trimesh.Trimesh(vertices = name.v, faces = model.f).show(viewer = 'gl')
    mesh.show(viewer = 'gl')

[-0.06366303 -0.05453378  0.0523619   0.00803254  0.0036163  -0.04558565
 -0.07479777  0.00206427 -0.04035823  0.04482574 -0.0310239   0.0531057
  0.02521082 -0.03508798 -0.0386354 ]
(5023, 3)


In [ ]:
variables = [i for i in dir(mesh) if not callable(i)]
print(variables)

In [6]:
print(mesh.centroid)

[-0.0003605 -0.0204175 -0.0415275]
